In [14]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv(os.path.join(os.getcwd(), '..'))

GOOGLE_API_KEY="dfafdasf"
genai.configure(api_key=GOOGLE_API_KEY)

In [15]:
GOOGLE_API_KEY

'dfafdasf'

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-flash")
response = model.generate_content("How are you")
response

In [17]:
from google.api_core.exceptions import InvalidArgument

try:
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model.generate_content("How are you")
except InvalidArgument:
    print("Invalid API key.")

Invalid API key.


In [5]:
# testing the documents and embeddings generated using the Punjab] Board of Revenue Act, 1957 pdf
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser


def load_split_pdf(pdf_filepath):
    loader = PDFMinerLoader(pdf_filepath)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(['\n', '\n\n', '.', ','])
    docs = splitter.split_documents(docs)
    print("splitted")
    return docs

In [11]:

def prepare_vdb_retriever(docs):
    texts = "\n\n".join([doc.page_content for doc in docs])
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    if not os.path.exists("vdb_chroma_embeddings_BOR_1957"):
        print("Embeddings already exists. Skipping creating/saving again.")
        vdb = Chroma.from_texts(texts, embeddings, persist_directory="./vdb_chroma_embeddings_BOR_1957")
    else:
        vdb = Chroma(embedding_function=embeddings, persist_directory="./vdb_chroma_embeddings_BOR_1957")

    vdb_retriever = vdb.as_retriever()
    print("VDB created")

    return vdb_retriever


In [6]:
docs = load_split_pdf(r"resources\books\BOARD OF REVENUE ACT 1957.pdf")
docs

splitted


[Document(metadata={'source': 'resources\\books\\BOARD OF REVENUE ACT 1957.pdf'}, page_content='THE PUNJAB BOARD OF REVENUE ACT, 1957   \n\nPage 1 of 5 \n\nTHE PUNJAB BOARD OF REVENUE ACT, 1957 \n(W.P. Act XI of 1957) \n\nC O N T E N T S \n\nSECTIONS \n\n1. \n\n2. \n\n3. \n\n4. \n\n5. \n\n6. \n\n7. \n\n8. \n\n9. \n\n10. \n\n11. \n\nShort title, extent and commencement. \n\nDefinitions. \n\nConstitution of a Board of Revenue. \n\nSuperintendence and control of Revenue Officers and Revenue Courts. \n\nPowers of the Board. \n\nConduct of business. \n\nRevision of orders by the Board. \n\nReview of orders by the Board. \n\nPower to make rules. \n\nSavings and validation. \n\nRepeal of  West Pakistan Ordinance XII of 1956 and continuance of action  taken \nthereunder. \n\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n\x0cTHE PUNJAB BOARD OF REVENUE ACT, 1957   \n\nPage 2 of 5 \n\n1THE 2[PUNJAB] BOARD OF REVENUE ACT, 1957 \n(W.P. Act XI of 1957) \n\n[28 February 1957] \n\nAn Act to provide for t

In [12]:
vdb_retriever = prepare_vdb_retriever(docs)
vdb_retriever

VDB created


VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000285C6BC8230>)

In [14]:
vdb_retriever.invoke("what is THE PUNJAB BOARD OF REVENUE ACT, 1957 all about")

[Document(page_content='\n'),
 Document(page_content=' '),
 Document(page_content=' '),
 Document(page_content=' ')]